In [7]:
import csv
import json
import unicodedata
import numpy as np
import networkx as nx
import requests
import pandas as pd
from pattern import web
import matplotlib.pyplot as plt
import re
# set some nicer defaults for matplotlib
from matplotlib import rcParams
from bs4 import BeautifulSoup

In [8]:
def find_str(s, char):
    index = 0
    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return index

            index += 1

    return -1

In [11]:
def get_poll_xml1_draft(poll_id):
    #url = "http://www.nhl.com/scores/htmlreports/20102011/GS0%i.HTM" % int(poll_id)
    url = "http://www.hockey-reference.com/draft/NHL_%i_entry.html" % int(poll_id)
    
    return requests.get(url).text

In [13]:
#http://www.hockey-reference.com/players/c/crosbsi01.html
def get_poll_xml1_player(poll_id):
    #url = "http://www.nhl.com/scores/htmlreports/20102011/GS0%i.HTM" % int(poll_id)
    url = "http://www.hockey-reference.com" + poll_id

    return requests.get(url).text

In [15]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False


In [16]:
soup = BeautifulSoup(get_poll_xml1_draft(2009))
play_list=[]

for year in range(2009,(2009+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:212]

playerdata=[]


for i in range(len(play_list)):
    print i
    soup2 = BeautifulSoup(get_poll_xml1_player(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    #draft=strsoup2[(draftPos+7):(draftPos+10)]
    if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+8)])
    elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+9)])
    else:
        draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2009=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2009.to_csv('dfplayer2009.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [17]:
dfplayer2009.to_csv('Data/draft_year_player/dfplayer2009.csv')    

In [9]:
for year in range(2011,2012):
    print year
    csv_file='Data/draft_year_player/dfplayer'+str(year)+'.csv'
    
    dataplayer=pd.read_csv(csv_file,index_col=False)

    for elem in dataplayer['url']:
        print elem
        url = "http://www.hockey-reference.com"+elem
        
        page=requests.get(url).text
        indexLoc= [i for i,x in enumerate(url) if x == '/']
        indexDot= [i for i,x in enumerate(url) if x == '.']
        pagesoap=BeautifulSoup(page)
        strsoap=str(pagesoap)
        posseStr=(find_str(strsoap, "Defensive Zone Faceoffs / (Offensive Zone Faceoffs + Defensive Zone Faceoffs),<br />that took place while on the ice"))

        posStr=(find_str(strsoap, "Awards</th>\n</tr>\n</thead>\n<tbody>"))
        posEnd=(find_str(strsoap, "</tr>\n</tfoot>\n</table>\n</div>\n</div>\n</div><!-- div.table_wrapper -->\n<!-- sr_mobile_1 -->"))
        posseEnd=(find_str(strsoap, " <h2>NHL Miscellaneous</h2>"))

        strlimitPos=strsoap[(posseStr+116):posseEnd]
        strlimit=strsoap[(posStr+34):posEnd]

        age=['age']
        season=['season']
        corsi_for=['corsi_for']
        corsi_against=['corsi_against']
        corsi_pct=['corsi_pct']
        corsi_rel_pct=['corsi_rel_pct']
        fenwick_for=['fenwick_for']
        fenwick_against=['fenwick_against']
        fenwick_pct=['fenwick_pct']
        fenwick_rel_pct=['fenwick_rel_pct']
        on_ice_shot_pct=['on_ice_shot_pct']
        on_ice_sv_pct=['on_ice_sv_pct']
        teams=['team']
        pdo=['pdo']
        zs_offense_pct=['zs_offense_pct']
        zs_defense_pct=['zs_defense_pct']

        # for m in re.finditer('season', strlimitPos):     
        # #     if strlimitPos[(m.end()+3):(m.end()+5)]!='20' and strlimitPos[(m.end()+3):(m.end()+5)]!='19':
        # #         break

        # #     else:
        #     print strlimitPos[(m.end()+3):(m.end()+10)]

        countYear=0
        for m in re.finditer('season', strlimitPos):     
            if strlimitPos[(m.end()+3):(m.end()+5)]!='20' and strlimitPos[(m.end()+3):(m.end()+5)]!='19':
        #         print strlimitPos[(m.end()+3):(m.end()+10)]
                break

            else:
                countYear=countYear+1
                season.append(strlimitPos[(m.end()+3):(m.end()+10)])

        #for m in re.finditer('team_id', strlimitPos):
#             if strlimitPos[(m.end()+3):(m.end()+5)]=='</':
#                 break
#             else:
            #teams.append(strlimitPos[(m.end()+10):(m.end()+25)])
        
        for m in re.finditer('age', strlimitPos):
            if strlimitPos[(m.end()+3):(m.end()+5)]=='</':
                break
            else:
                age.append(strlimitPos[(m.end()+3):(m.end()+5)])
                teams.append(strlimitPos[(m.end()+65):(m.end()+68)])

        for m in re.finditer('corsi_for', strlimitPos):
            if strlimitPos[(m.end()+4)]=='<':
                corsi_for.append(strlimitPos[(m.end()+3):(m.end()+4)])

            elif strlimitPos[(m.end()+5)]=='<':
                corsi_for.append(strlimitPos[(m.end()+3):(m.end()+5)])

            elif strlimitPos[(m.end()+6)]=='<':
                corsi_for.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                corsi_for.append(strlimitPos[(m.end()+3):(m.end()+7)])

        for m in re.finditer('corsi_against', strlimitPos):
            if strlimitPos[(m.end()+4)]=='<':
                corsi_against.append(strlimitPos[(m.end()+3):(m.end()+4)])

            elif strlimitPos[(m.end()+5)]=='<':
                corsi_against.append(strlimitPos[(m.end()+3):(m.end()+5)])

            elif strlimitPos[(m.end()+6)]=='<':
                corsi_against.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                corsi_against.append(strlimitPos[(m.end()+3):(m.end()+7)])

        for m in re.finditer('corsi_pct', strlimitPos):
            if strlimitPos[(m.end()+6)]=='<':
                corsi_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                corsi_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])

        for m in re.finditer('corsi_rel_pct', strlimitPos):
            if strlimitPos[(m.end()+6)]=='<':
                corsi_rel_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                corsi_rel_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])

        for m in re.finditer('fenwick_for', strlimitPos):
            if strlimitPos[(m.end()+4)]=='<':
                fenwick_for.append(strlimitPos[(m.end()+3):(m.end()+4)])
            elif strlimitPos[(m.end()+5)]=='<':
                fenwick_for.append(strlimitPos[(m.end()+3):(m.end()+5)])
            elif strlimitPos[(m.end()+6)]=='<':
                fenwick_for.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                fenwick_for.append(strlimitPos[(m.end()+3):(m.end()+7)])

        for m in re.finditer('fenwick_against', strlimitPos):
            if strlimitPos[(m.end()+4)]=='<':
                fenwick_against.append(strlimitPos[(m.end()+3):(m.end()+4)])
            elif strlimitPos[(m.end()+5)]=='<':
                fenwick_against.append(strlimitPos[(m.end()+3):(m.end()+5)])
            elif strlimitPos[(m.end()+6)]=='<':
                fenwick_against.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                fenwick_against.append(strlimitPos[(m.end()+3):(m.end()+7)])        


        for m in re.finditer('fenwick_pct', strlimitPos):
            if strlimitPos[(m.end()+6)]=='<':
                fenwick_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                fenwick_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])        

        for m in re.finditer('fenwick_rel_pct', strlimitPos):
            if strlimitPos[(m.end()+6)]=='<':
                fenwick_rel_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                fenwick_rel_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])        


        for m in re.finditer('on_ice_shot_pct', strlimitPos):
            if strlimitPos[(m.end()+6)]=='<':
                on_ice_shot_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                on_ice_shot_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])

        for m in re.finditer('on_ice_sv_pct', strlimitPos):
            if strlimitPos[(m.end()+6)]=='<':
                on_ice_sv_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                on_ice_sv_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])        

        for m in re.finditer('pdo', strlimitPos):
            if strlimitPos[(m.end()+6)]=='<':
                pdo.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                pdo.append(strlimitPos[(m.end()+3):(m.end()+7)])        

        for m in re.finditer('zs_offense_pct', strlimitPos):
            if strlimitPos[(m.end()+6)]=='<':
                zs_offense_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                zs_offense_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])        

        for m in re.finditer('zs_defense_pct', strlimitPos):
            if strlimitPos[(m.end()+6)]=='<':
                zs_defense_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
            else:
                zs_defense_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])         

    #     print countYear
        for ith in range(len(teams)):
            for elem in teams[ith]:
                if elem.isupper()==False:
                    teams[ith]="TOT"
        matrix=[]      
        
        matrix.append(season[:(countYear+1)])
        matrix.append(teams[:(countYear+1)])
        matrix.append(age[:(countYear+1)])
        matrix.append(corsi_for[:(countYear+1)])
        matrix.append(corsi_against[:(countYear+1)])
        matrix.append(corsi_pct[:(countYear+1)])
        matrix.append(corsi_rel_pct[:(countYear+1)])
        matrix.append(fenwick_for[:(countYear+1)])
        matrix.append(fenwick_against[:(countYear+1)])
        matrix.append(fenwick_pct[:(countYear+1)])
        matrix.append(fenwick_rel_pct[:(countYear+1)])

        matrix.append(on_ice_shot_pct[:(countYear+1)])
        matrix.append(on_ice_sv_pct[:(countYear+1)])
        matrix.append(pdo[:(countYear+1)])

        matrix.append(zs_offense_pct[:(countYear+1)])
        matrix.append(zs_defense_pct[:(countYear+1)])
        newmatrix=[list(i) for i in zip(*matrix)]
        
        #print 'debug'
        #print matrix
        newdf=pd.DataFrame(newmatrix)

        newdf.columns = newdf.iloc[0]
        newdf = newdf[1:]
        #print age
        namecsv=url[(indexLoc[-1]+1):(indexDot[-1])]+".csv"
    #     newdf=newdf[(newdf.season == '2004-05')|(newdf.season == '2005-06')|(newdf.season == '2006-07')|
    #                 (newdf.season == '2007-08')|(newdf.season == '2008-09')|(newdf.season == '2009-10')|
    #                 (newdf.season == '2010-11')|(newdf.season == '2011-12')|(newdf.season == '2012-13')|
    #                 (newdf.season == '2013-14')|(newdf.season == '2014-15')|(newdf.season == '2015-16')]

        namecsv="Data/PlayerData/"+str(year)+"/posession/"+url[(indexLoc[-1]+1):(indexDot[-1])]+"_pos_"+".csv"

        newdf.to_csv(namecsv)  
        age=['age']
        #teams=['team']
        games_played=['games_played']
        goals=['goals']
        assists=['assists']
        points=['points']
        plus_minus=['plus_minus']
        pen_min=['pen_min']
        goals_ev=['goals_ev']
        goals_pp=['goals_pp']
        goals_sh=['goals_sh']
        goals_gw=['goals_gw']
        assists_ev=['assists_ev']
        assists_pp=['assists_pp']
        assists_sh=['assists_sh']
        shots=['shots']
        shot_pct=['shot_pct']
        
        shots_attempted=['shots_attempted']
        time_on_ice=['time_on_ice']
        time_on_ice_avg=['time_on_ice_avg']
        faceoff_wins=[]
        faceoff_losses=[]
        faceoff_percentage=[]
        hits=['hits']
        blocks=['blocks']
        takeaways=['takeaways']
        giveaways=['giveaways']
        award_summary=['award_summary']
        teams=['team']
        countYear=0
        season1=['season']
        for m in re.finditer('season', strlimit):  
            #print strlimit[m.end():(m.end()+10)]
            #season1.append(strlimit[(m.end()+14):(m.end()+21)])
            if strlimit[(m.end()+14):(m.end()+16)]!='20' and strlimit[(m.end()+14):(m.end()+16)]!='19':
                break

            else:
                countYear=countYear+1
                season1.append(strlimit[(m.end()+14):(m.end()+21)])
                #season.append(strlimit[(m.end()+3):(m.end()+10)])
        
        #print 'debug'
        for m in re.finditer('-stat="age">', strlimit):
            #print strlimit[(m.end()):(m.end()+2)]
            if strlimit[(m.end()):(m.end()+2)]=='</':
                break
            else:
                age.append(strlimit[(m.end()):(m.end()+2)])
                #teams.append(strlimit[(m.end()+65):(m.end()+68)])
        
        #print 'debug'
        teams=['TOT']
        for m in re.finditer('data-stat="team_id">', strlimit):
            #print strlimit[(m.end()):(m.end()+10)]
            #print strlimit[(m.end()+16):(m.end()+19)]
            #if strlimit[(m.end()):(m.end()+2)]=='</t':
            #    break
            #else:
                #age.append(strlimit[(m.end()):(m.end()+2)])
            
            if strlimit[(m.end()):(m.end()+3)]=='TOT':
                teams.append(strlimit[(m.end()):(m.end()+3)])
            else:
                if strlimit[(m.end()+16):(m.end()+17)].isupper()==True:
                    teams.append(strlimit[(m.end()+16):(m.end()+19)])

        for m in re.finditer('games_played', strlimit):
            #print strlimit[(m.end()):(m.end()+10)]
            if strlimit[(m.end()+2):(m.end()+10)]=='<strong>':
                games_played.append(strlimit[(m.end()+10):(m.end()+12)])
            elif strlimit[(m.end()+3)]=='<':
                games_played.append(strlimit[(m.end()+2):(m.end()+3)])
            else:
                games_played.append(strlimit[(m.end()+2):(m.end()+4)])

        for m in re.finditer('goals_ev', strlimit):
            if strlimit[(m.end()+2):(m.end()+10)]=='<strong>':
                goals_ev.append(strlimit[(m.end()+10):(m.end()+12)])
            elif strlimit[(m.end()+3)]=='<':
                goals_ev.append(strlimit[(m.end()+2):(m.end()+3)])
            else:
                goals_ev.append(strlimit[(m.end()+2):(m.end()+4)])

        for m in re.finditer('goals_pp', strlimit):
            if strlimit[(m.end()+2):(m.end()+10)]=='<strong>':
                goals_pp.append(strlimit[(m.end()+10):(m.end()+12)])
            elif strlimit[(m.end()+3)]=='<':
                goals_pp.append(strlimit[(m.end()+2):(m.end()+3)])
            else:
                goals_pp.append(strlimit[(m.end()+2):(m.end()+4)])


        for m in re.finditer('goals_sh', strlimit):
            if strlimit[(m.end()+2):(m.end()+10)]=='<strong>':
                goals_sh.append(strlimit[(m.end()+10):(m.end()+12)])
            elif strlimit[(m.end()+3)]=='<':
                goals_sh.append(strlimit[(m.end()+2):(m.end()+3)])
            else:
                goals_sh.append(strlimit[(m.end()+2):(m.end()+4)])

        for m in re.finditer('goals_gw', strlimit):
            if strlimit[(m.end()+2):(m.end()+10)]=='<strong>':
                goals_gw.append(strlimit[(m.end()+10):(m.end()+12)])
            elif strlimit[(m.end()+3)]=='<':
                goals_gw.append(strlimit[(m.end()+2):(m.end()+3)])
            else:
                goals_gw.append(strlimit[(m.end()+2):(m.end()+4)])


        for m in re.finditer('assists_ev', strlimit):
            if strlimit[(m.end()+2):(m.end()+10)]=='<strong>':
                assists_ev.append(strlimit[(m.end()+10):(m.end()+12)])
            elif strlimit[(m.end()+3)]=='<':
                assists_ev.append(strlimit[(m.end()+2):(m.end()+3)])
            else:
                assists_ev.append(strlimit[(m.end()+2):(m.end()+4)])

        for m in re.finditer('assists_pp', strlimit):
            if strlimit[(m.end()+2):(m.end()+10)]=='<strong>':
                assists_pp.append(strlimit[(m.end()+10):(m.end()+12)])
            elif strlimit[(m.end()+3)]=='<':
                assists_pp.append(strlimit[(m.end()+2):(m.end()+3)])
            else:
                assists_pp.append(strlimit[(m.end()+2):(m.end()+4)])

        for m in re.finditer('assists_sh', strlimit):
            if strlimit[(m.end()+3)]=='<':
                assists_sh.append(strlimit[(m.end()+2):(m.end()+3)])
            else:
                assists_sh.append(strlimit[(m.end()+2):(m.end()+4)])

        for m in re.finditer('shot_pct', strlimit):
            if strlimit[(m.end()+5)]=='<':
                shot_pct.append(strlimit[(m.end()+2):(m.end()+5)])
            else:
                shot_pct.append(strlimit[(m.end()+2):(m.end()+6)])

        for m in re.finditer('shots_attempted', strlimit):
            if strlimit[(m.end()+2):(m.end()+6)]=='</td':
                shots_attempted.append('')
            elif strlimit[(m.end()+3)]=='<':
                shots_attempted.append(strlimit[(m.end()+2):(m.end()+3)])
            elif strlimit[(m.end()+4)]=='<':
                shots_attempted.append(strlimit[(m.end()+2):(m.end()+4)])
            elif strlimit[(m.end()+5)]=='<':
                shots_attempted.append(strlimit[(m.end()+2):(m.end()+5)])
            else:
                shots_attempted.append(strlimit[(m.end()+2):(m.end()+6)])
        for m in re.finditer('time_on_ice', strlimit):
            time_on_ice.append(strlimit[(m.end()+2):(m.end()+4)])    

        for m in re.finditer('time_on_ice_avg', strlimit):
            if strlimit[(m.end()+6)]=='<':
                time_on_ice_avg.append(strlimit[(m.end()+2):(m.end()+6)]) 
            else:
                time_on_ice_avg.append(strlimit[(m.end()+2):(m.end()+7)])  


        for m in re.finditer('hits', strlimit):
            if strlimit[(m.end()+2):(m.end()+7)]=='</td>':
                hits.append('')
            elif strlimit[(m.end()+3)]=='<':
                hits.append(strlimit[(m.end()+2):(m.end()+3)])
            elif strlimit[(m.end()+4)]=='<':
                hits.append(strlimit[(m.end()+2):(m.end()+4)])
            else:
                hits.append(strlimit[(m.end()+2):(m.end()+5)])

        for m in re.finditer('blocks', strlimit):
            if strlimit[(m.end()+2):(m.end()+7)]=='</td>':
                blocks.append('')
            elif strlimit[(m.end()+3)]=='<':
                blocks.append(strlimit[(m.end()+2):(m.end()+3)])
            elif strlimit[(m.end()+4)]=='<':
                blocks.append(strlimit[(m.end()+2):(m.end()+4)])
            else:
                blocks.append(strlimit[(m.end()+2):(m.end()+5)])

        for m in re.finditer('takeaways', strlimit):
            if strlimit[(m.end()+2):(m.end()+7)]=='</td>':
                takeaways.append('')
            elif strlimit[(m.end()+3)]=='<':
                takeaways.append(strlimit[(m.end()+2):(m.end()+3)])
            elif strlimit[(m.end()+4)]=='<':
                takeaways.append(strlimit[(m.end()+2):(m.end()+4)])
            else:
                takeaways.append(strlimit[(m.end()+2):(m.end()+5)])


        for m in re.finditer('giveaways', strlimit):
            if strlimit[(m.end()+2):(m.end()+7)]=='</td>':
                giveaways.append('')
            elif strlimit[(m.end()+3)]=='<':
                giveaways.append(strlimit[(m.end()+2):(m.end()+3)])
            elif strlimit[(m.end()+4)]=='<':
                giveaways.append(strlimit[(m.end()+2):(m.end()+4)])
            else:
                giveaways.append(strlimit[(m.end()+2):(m.end()+5)])

        for m in re.finditer('plus_minus', strlimit):
            if strlimit[(m.end()+3)]=='<':
                plus_minus.append(strlimit[(m.end()+2):(m.end()+3)])
            elif strlimit[(m.end()+4)]=='<':
                plus_minus.append(strlimit[(m.end()+2):(m.end()+4)])
            elif strlimit[(m.end()+5)]=='<':
                plus_minus.append(strlimit[(m.end()+2):(m.end()+5)])
            else:
                plus_minus.append(strlimit[(m.end()+2):(m.end()+6)])

        for m in re.finditer('pen_min', strlimit):
            if strlimit[(m.end()+3)]=='<':
                pen_min.append(strlimit[(m.end()+2):(m.end()+3)])
            elif strlimit[(m.end()+4)]=='<':
                pen_min.append(strlimit[(m.end()+2):(m.end()+4)])
            elif strlimit[(m.end()+5)]=='<':
                pen_min.append(strlimit[(m.end()+2):(m.end()+5)])
            else:
                pen_min.append(strlimit[(m.end()+2):(m.end()+6)])

#         for ith in range(len(teams)):
#             for elem in teams[ith]:
#                 if elem.isupper()==False:
#                     teams[ith]="TOT"
                    
            
        matrix=[]
        matrix.append(season1[:(countYear+1)])
        matrix.append(teams[:(countYear+1)])
        matrix.append(age[:(countYear+1)])
        matrix.append(games_played[:(countYear+1)])
        matrix.append(pen_min[:(countYear+1)])
        matrix.append(plus_minus[:(countYear+1)])
        matrix.append(goals_ev[:(countYear+1)])
        matrix.append(goals_pp[:(countYear+1)])
        matrix.append(goals_sh[:(countYear+1)])
        matrix.append(goals_gw[:(countYear+1)])

        matrix.append(assists_ev[:(countYear+1)])
        matrix.append(assists_pp[:(countYear+1)])
        matrix.append(assists_sh[:(countYear+1)])

        matrix.append(shot_pct[:(countYear+1)])
        matrix.append(shots_attempted[:(countYear+1)])

        matrix.append(shot_pct[:(countYear+1)])
        matrix.append(shots_attempted[:(countYear+1)])
        matrix.append(time_on_ice_avg[:(countYear+1)])
        matrix.append(hits[:(countYear+1)])

        matrix.append(blocks[:(countYear+1)])
        matrix.append(takeaways[:(countYear+1)])
        matrix.append(giveaways[:(countYear+1)])
    #     print countYear
        newmatrix=[list(i) for i in zip(*matrix)]

        newdf=pd.DataFrame(newmatrix)

        newdf.columns = newdf.iloc[0]
        newdf = newdf[1:]

        namecsv=url[(indexLoc[-1]+1):(indexDot[-1])]+".csv"
    #     newdf=newdf[(newdf.season == '2004-05')|(newdf.season == '2005-06')|(newdf.season == '2006-07')|
    #                 (newdf.season == '2007-08')|(newdf.season == '2008-09')|(newdf.season == '2009-10')|
    #                 (newdf.season == '2010-11')|(newdf.season == '2011-12')|(newdf.season == '2012-13')|
    #                 (newdf.season == '2013-14')|(newdf.season == '2014-15')|(newdf.season == '2015-16')]

        namecsv="Data/PlayerData/"+str(year)+"/standard/"+url[(indexLoc[-1]+1):(indexDot[-1])]+"_strd_"+".csv"

        newdf.to_csv(namecsv)  
        

2011
/players/n/nugenry01.html


/Users/takehiromatsuzawa/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


/players/l/landega01.html
/players/h/huberjo01.html
/players/l/larssad01.html
/players/s/stromry01.html
/players/z/zibanmi01.html
/players/s/scheima01.html
/players/c/coutuse01.html
/players/h/hamildo01.html
/players/b/brodijo01.html
/players/s/siemedu01.html
/players/m/murphry01.html
/players/b/baertsv01.html
/players/o/oleksja01.html
/players/m/millejt01.html
/players/a/armiajo01.html
/players/b/beaulna01.html
/players/m/mcneima01.html
/players/k/klefbos01.html
/players/m/murphco02.html
/players/n/noesest01.html
/players/b/biggsty01.html
/players/m/morrojo02.html
/players/p/puempma01.html
/players/p/percyst01.html
/players/d/danauph01.html
/players/n/namesvl01.html
/players/p/phillza01.html
/players/j/jenseni01.html
/players/r/rakelri01.html
/players/m/musilda01.html
/players/r/rattity01.html
/players/g/grimaro01.html
/players/m/mayfisc01.html
/players/j/jurcoto01.html
/players/c/clendad01.html
/players/j/jennebo01.html
/players/h/hellbma01.html
/players/g/gibsojo02.html
/players/k/k